# Library

In [105]:
import os
from typing import List, Dict, Tuple
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pywt

# Data

In [106]:
data_path = "../../../data/" # 본인 작업환경에 맞게 변경

train = pd.read_csv(path+"train.csv").assign(_type="train")
test = pd.read_csv(path+"test.csv").assign(_type="test")
submission = pd.read_csv(path+"test.csv")
df = pd.concat([train, test], axis=0)

# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")

100%|██████████| 107/107 [00:05<00:00, 20.99it/s]


# Null

In [107]:
# 결측치 처리 - 칼럼 정리 

# 결측치 확인 및 처리를 위해 train, test로 분리 
train_df = df[df['_type'] == 'train']
test_df = df[df['_type'] == 'test']

# 결측치 많은 칼럼 확인
missing_values = train_df.isnull().sum() 
columns_with_many_nulls = missing_values[missing_values > 4000].index # train에서 결측값이 4000개 이상인 열
nan_cols = test_df.iloc[:,2:].isnull().sum()[test_df.iloc[:,2:].isnull().sum() > 2000].index # test에서 결측값이 2000개 이상인 열
columns_with_many_nulls = set(nan_cols).union(set(columns_with_many_nulls))

# 결측치 많은 칼럼 제거 후 train, test 다시 병합
train_df = train_df.drop(columns_with_many_nulls,axis=1)
test_df = test_df.drop(columns_with_many_nulls,axis=1)
df= pd.concat([train_df, test_df], axis = 0)

In [108]:
# 결측치 처리 - 행 정리(1)

df_new = df.drop(["ID", "target", "_type"], axis = 1)

# 결측치 확인
missing_data = df_new.isnull().sum()

# 결측치 비율 확인
missing_ratio = df_new.isnull().mean()


display(missing_data.sort_values(ascending=False))
display(missing_ratio.sort_values(ascending=False))

hourly_market-data_open-interest_gate_io_btc_usd_open_interest                    510
hourly_market-data_liquidations_bitfinex_all_symbol_long_liquidations             150
hourly_market-data_liquidations_bitfinex_all_symbol_short_liquidations            150
hourly_market-data_liquidations_bitfinex_all_symbol_long_liquidations_usd         150
hourly_market-data_liquidations_bitfinex_all_symbol_short_liquidations_usd        150
                                                                                 ... 
hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd       0
hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd      0
hourly_network-data_fees-transaction_fees_transaction_median                        0
hourly_network-data_fees-transaction_fees_transaction_median_usd                    0
hourly_market-data_open-interest_htx_global_btc_usdt_open_interest                  0
Length: 205, dtype: int64

hourly_market-data_open-interest_gate_io_btc_usd_open_interest                    0.044148
hourly_market-data_liquidations_bitfinex_all_symbol_long_liquidations             0.012985
hourly_market-data_liquidations_bitfinex_all_symbol_short_liquidations            0.012985
hourly_market-data_liquidations_bitfinex_all_symbol_long_liquidations_usd         0.012985
hourly_market-data_liquidations_bitfinex_all_symbol_short_liquidations_usd        0.012985
                                                                                    ...   
hourly_market-data_liquidations_all_exchange_all_symbol_long_liquidations_usd     0.000000
hourly_market-data_liquidations_all_exchange_all_symbol_short_liquidations_usd    0.000000
hourly_network-data_fees-transaction_fees_transaction_median                      0.000000
hourly_network-data_fees-transaction_fees_transaction_median_usd                  0.000000
hourly_market-data_open-interest_htx_global_btc_usdt_open_interest                0.000000

In [109]:
# 1. 결측치 처리 - 행 정리(2) 앞/뒤 보간법 (forward/backward fill)

# df_ffill = df_new.fillna(method='ffill')  # 앞의 값을 사용하여 결측치 채움
# df_bfill = df_new.fillna(method='bfill')  # 뒤의 값을 사용하여 결측치 채움

# 2. 결측치 처리 - 선형 보간법 (linear interpolation)
df_interpolated = df_new.interpolate(method='linear')

# 3. 결측치 처리 - 평균값으로 대체
# df_mean_filled = df_new.fillna(eda_df.mean())

df.update(df_interpolated)

# 결측치 처리 후 확인
print(df_interpolated.isnull().sum())

hourly_market-data_liquidations_gate_io_all_symbol_long_liquidations          0
hourly_market-data_liquidations_gate_io_all_symbol_short_liquidations         0
hourly_market-data_liquidations_gate_io_all_symbol_long_liquidations_usd      0
hourly_market-data_liquidations_gate_io_all_symbol_short_liquidations_usd     0
hourly_market-data_funding-rates_bybit_funding_rates                          0
                                                                             ..
hourly_market-data_liquidations_htx_global_btc_usdt_long_liquidations         0
hourly_market-data_liquidations_htx_global_btc_usdt_short_liquidations        0
hourly_market-data_liquidations_htx_global_btc_usdt_long_liquidations_usd     0
hourly_market-data_liquidations_htx_global_btc_usdt_short_liquidations_usd    0
hourly_market-data_open-interest_htx_global_btc_usdt_open_interest            0
Length: 205, dtype: int64


# Feature Engineering

## Received Baseline

In [110]:
# 불필요한 접두사 제거 
df.columns = df.columns.str.replace('hourly_market-data_|hourly_network-data_', '', regex=True)
display(df.columns)

Index(['ID', 'target', '_type',
       'liquidations_gate_io_all_symbol_long_liquidations',
       'liquidations_gate_io_all_symbol_short_liquidations',
       'liquidations_gate_io_all_symbol_long_liquidations_usd',
       'liquidations_gate_io_all_symbol_short_liquidations_usd',
       'funding-rates_bybit_funding_rates',
       'liquidations_htx_global_all_symbol_long_liquidations',
       'liquidations_htx_global_all_symbol_short_liquidations',
       ...
       'taker-buy-sell-stats_binance_taker_buy_sell_ratio',
       'liquidations_huobi_global_btc_usdt_long_liquidations',
       'liquidations_huobi_global_btc_usdt_short_liquidations',
       'liquidations_huobi_global_btc_usdt_long_liquidations_usd',
       'liquidations_huobi_global_btc_usdt_short_liquidations_usd',
       'liquidations_htx_global_btc_usdt_long_liquidations',
       'liquidations_htx_global_btc_usdt_short_liquidations',
       'liquidations_htx_global_btc_usdt_long_liquidations_usd',
       'liquidations_htx_g

In [111]:
# 게시글에 있는 베이스라인 코드의 피처 엔지니어링 부분과 동일 
# (eda 에서 파악한 차이와 차이의 음수, 양수 여부를 새로운 피쳐로 생성)
def make_baseline_extra_feature(df): 

    df = df.assign(
    liquidation_diff=df["liquidations_all_exchange_all_symbol_long_liquidations"] - df["liquidations_all_exchange_all_symbol_short_liquidations"],
    liquidation_usd_diff=df["liquidations_all_exchange_all_symbol_long_liquidations_usd"] - df["liquidations_all_exchange_all_symbol_short_liquidations_usd"],
    volume_diff=df["taker-buy-sell-stats_all_exchange_taker_buy_volume"] - df["taker-buy-sell-stats_all_exchange_taker_sell_ratio"],
    liquidation_diffg=np.sign(df["liquidations_all_exchange_all_symbol_long_liquidations"] - df["liquidations_all_exchange_all_symbol_short_liquidations"]),
    liquidation_usd_diffg=np.sign(df["liquidations_all_exchange_all_symbol_long_liquidations_usd"] - df["liquidations_all_exchange_all_symbol_short_liquidations_usd"]),
    volume_diffg=np.sign(df["taker-buy-sell-stats_all_exchange_taker_buy_volume"] - df["taker-buy-sell-stats_all_exchange_taker_sell_ratio"]),
    buy_sell_volume_ratio=df["taker-buy-sell-stats_all_exchange_taker_buy_volume"] / (df["taker-buy-sell-stats_all_exchange_taker_sell_ratio"] + 1))

    return df

In [112]:
# 적용
df = make_baseline_extra_feature(df)
df['liquidation_diff'] # 확인

0        0.012000
1       -0.712000
2        0.000000
3        0.593000
4        0.361000
           ...   
11547    0.466500
11548    6.431208
11549   -3.419327
11550   -0.853000
11551   -1.569674
Name: liquidation_diff, Length: 11552, dtype: float64

In [113]:
def shift_feature(
    df: pd.DataFrame,
    conti_cols: List[str],
    intervals: List[int],
) -> List[pd.Series]:
    """
    연속형 변수의 shift feature 생성
    Args:
        df (pd.DataFrame)
        conti_cols (List[str]): continuous colnames
        intervals (List[int]): shifted intervals
    Return:
        List[pd.Series]
    """
    df_shift_dict = [
        df[conti_col].shift(interval).rename(f"{conti_col}_{interval}")
        for conti_col in conti_cols
        for interval in intervals
    ]
    return df_shift_dict

# 지수 이동 평균
def EMA(df, col, span=2):
    return df[col].ewm(span=span).mean()

# Wavlet Transform
def WT(df, col, wavelet='db5', th=0.6):
    signal = df[col].values
    th = th*np.nanmax(signal)
    coef = pywt.wavedec(signal, wavelet, mode="per" )
    coef[1:] = (pywt.threshold(i, value=th, mode="soft" ) for i in coef[1:])
    reconstructed = pywt.waverec(coef, wavelet, mode="per" )
    return reconstructed

## Date Feature

In [114]:
# 날짜 관련 생성
def make_date_features(df: pd.DataFrame, date_column: str) -> Tuple[pd.DataFrame]:
    """
    입력된 데이터프레임의 특정 날짜 열을 기준으로 연도, 월, 주, 요일, 시간을 추출하여 새로운 피처로 추가.
    Args:
        df (pd.DataFrame): 날짜 컬럼을 포함하는 데이터프레임.
        date_column (str): 날짜 정보가 담긴 열 이름.
    Returns:
        pd.DataFrame: 날짜 관련 피처가 추가된 데이터프레임 반환.
    """
    df[date_column] = pd.to_datetime(df[date_column])
    df['year'] = df[date_column].dt.year  # 연도
    df['month'] = df[date_column].dt.month  # 월 
    df['week'] = df[date_column].dt.isocalendar().week  # 주
    df['day_of_week'] = df[date_column].dt.dayofweek  # 요일 
    df['hour'] = df[date_column].dt.hour  # 시간 
    return df

## Diff and Change of Open-Interest

In [115]:
# 변동성, 차분 피처 생성 함수 
def make_diff_change_feature(
    df: pd.DataFrame, 
    columns_list: List[str]
) -> pd.DataFrame:
    """
    주어진 변수들에 대한 변동성, 차분 피처를 생성하고, 결측값을 처리하는 함수.
    Args:
        df (pd.DataFrame): 데이터를 포함한 데이터프레임.
        columns_list (List[str]): 변동성과 차분 피처를 추가하고자 하는 열 이름 목록.
    Returns:
        pd.DataFrame: 변동성과 차분 피처가 추가된 데이터프레임.
    """  
    new_features = {}  # 새로 추가할 열을 저장할 딕셔너리
    for col in columns_list:
        if col in df.columns:
            pct_change_col = f'{col}_pct_change'
            diff_col = f'{col}_diff'
            new_features[pct_change_col] = df[col].pct_change(fill_method=None)
            new_features[diff_col] = df[col].diff()
        else:
            print(f"Error: Cannot find '{col}' column")
    new_features_df = pd.DataFrame(new_features, index=df.index) # 새 피처를 데이터프레임으로
    new_features_df = new_features_df.ffill().bfill()  # 결측값 처리 (고정: ffill, bfill)
    df = pd.concat([df, new_features_df], axis=1)  # 기존 데이터프레임에 추가
    return df

## Extra 7 Domain Related Features

In [116]:
# 1. 롱/숏 비율
def make_longshort_ratio_feature(
    df: pd.DataFrame, 
    long_col: str, 
    short_col: str
) -> pd.DataFrame:
    """
    롱/숏 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_col (str): 롱(liquidations) 데이터가 저장된 열 이름
        short_col (str): 숏(liquidations) 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 롱/숏 비율이 저장된 새로운 열이 추가된 데이터프레임
    """
    df['long_short_ratio'] = df[long_col] / (df[short_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [117]:
# 2. 청산/거래량 비율
def make_liquidation_to_volume_ratio_feature(
    df: pd.DataFrame, 
    long_col: str, 
    short_col: str, 
    buy_volume_col: str, 
    sell_volume_col: str
) -> pd.DataFrame:
    """
    청산/거래량 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_col (str): 롱(liquidations) 데이터가 저장된 열 이름
        short_col (str): 숏(liquidations) 데이터가 저장된 열 이름
        buy_volume_col (str): 매수 거래량이 저장된 열 이름
        sell_volume_col (str): 매도 거래량이 저장된 열 이름
    Returns:
        pd.DataFrame: 청산/거래량 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['liquidation_to_volume_ratio'] = (
        (df[long_col] + df[short_col]) / 
        (df[buy_volume_col] + df[sell_volume_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    )
    return df

In [118]:
# 3. 청산된 USD 롱/숏 비율
def make_liquidation_usd_ratio_feature(
    df: pd.DataFrame, 
    long_usd_col: str, 
    short_usd_col: str
) -> pd.DataFrame:
    """
    청산된 USD 롱/숏 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        long_usd_col (str): 롱(liquidations) USD 데이터가 저장된 열 이름
        short_usd_col (str): 숏(liquidations) USD 데이터가 저장된 열 이름 
    Returns:
        pd.DataFrame: 청산된 USD 롱/숏 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['liquidation_usd_ratio'] = df[long_usd_col] / (df[short_usd_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [119]:
# 4. 펀딩 비율과 롱/숏 포지션 차이 곱
def make_funding_rate_position_change_feature(
    df: pd.DataFrame, 
    funding_rate_col: str, 
    long_liquidations_col: str, 
    short_liquidations_col: str
) -> pd.DataFrame:
    """
    펀딩 비율과 롱/숏 포지션 차이를 곱하여 포지션 변화를 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        funding_rate_col (str): 펀딩 비율 데이터가 저장된 열 이름
        long_liquidations_col (str): 롱 청산 데이터가 저장된 열 이름
        short_liquidations_col (str): 숏 청산 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 펀딩 비율에 따른 포지션 변화를 저장한 새로운 열이 추가된 데이터프레임
    """
    df['funding_rate_position_change'] = df[funding_rate_col] * (
        df[long_liquidations_col] - df[short_liquidations_col]
    )
    return df


In [120]:
# 5. 프리미엄 갭과 프리미엄 인덱스의 차이
def make_premium_diff_feature(
    df: pd.DataFrame, 
    premium_gap_col: str, 
    premium_index_col: str
) -> pd.DataFrame:
    """
    프리미엄 갭과 프리미엄 인덱스의 차이를 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        premium_gap_col (str): 프리미엄 갭 데이터가 저장된 열 이름
        premium_index_col (str): 프리미엄 인덱스 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 프리미엄 갭과 프리미엄 인덱스의 차이를 저장한 새로운 열이 추가된 데이터프레임
    """
    df['premium_diff'] = df[premium_gap_col] - df[premium_index_col]
    return df

In [121]:
# 6. 해시레이트와 난이도 간의 비율
def make_hashrate_to_difficulty_feature(
    df: pd.DataFrame, 
    hashrate_col: str, 
    difficulty_col: str
) -> pd.DataFrame:
    """
    해시레이트와 난이도 간의 비율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        hashrate_col (str): 해시레이트 데이터가 저장된 열 이름
        difficulty_col (str): 난이도 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 해시레이트와 난이도 비율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['hashrate_to_difficulty'] = df[hashrate_col] / (df[difficulty_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

In [122]:
# 7. 공급 변화율
def make_supply_change_rate_feature(
    df: pd.DataFrame, 
    new_supply_col: str, 
    total_supply_col: str
) -> pd.DataFrame:
    """
    공급 변화율을 계산하는 함수
    Args:
        df (pd.DataFrame): 입력 데이터프레임
        new_supply_col (str): 새로운 공급량 데이터가 저장된 열 이름
        total_supply_col (str): 총 공급량 데이터가 저장된 열 이름
    Returns:
        pd.DataFrame: 공급 변화율을 저장한 새로운 열이 추가된 데이터프레임
    """
    df['supply_change_rate'] = df[new_supply_col] / (df[total_supply_col] + 1e-6)  # 분모가 0이 되는 것을 방지하기 위해 작은 값을 더함
    return df

## Rolling Features

In [123]:
def make_multiple_rolling_features(df, exclude_columns_list):
    """
    여러 윈도우 크기(6, 12, 24, 48)로 이동평균 피처를 생성하는 함수
    
    Parameters:
    - df: 입력 데이터프레임
    - exclude_columns_list: 이동평균을 적용하지 않을 열 이름이 담긴 리스트
    
    Returns:
    - 이동평균 피처가 추가된 데이터프레임
    """
    rolling_features = {}

    # 윈도우 크기별로 변수명 리스트 생성
    rolling_columns_dict = {window: [] for window in [6, 12, 24, 48]}

    for col in df.columns:
        if col not in exclude_columns_list:
            for window in [6, 12, 24, 48]:
                new_col_name = f'{col}_rolling_mean_{window}h'
                rolling_features[new_col_name] = df[col].rolling(window=window).mean()
                # 윈도우 크기에 맞는 리스트에 변수명 저장
                rolling_columns_dict[window].append(new_col_name)

    # 새로운 피처를 한 번에 추가
    rolling_features_df = pd.DataFrame(rolling_features, index=df.index)
    df = pd.concat([df, rolling_features_df], axis=1)
    return df

In [124]:
# 적용 

# 1. 날짜 피처 생성
df = make_date_features(df, 'ID') 

# 2. Open Interest 관련 변동성, 차분 피처 생성
imp_open_interest_columns = [
    'open-interest_all_exchange_all_symbol_open_interest',
    'open-interest_binance_btc_usd_open_interest',
    'open-interest_okx_btc_usdt_open_interest',
    'open-interest_deribit_btc_usd_open_interest',
    'open-interest_deribit_all_symbol_open_interest',
    'open-interest_bybit_btc_usdt_open_interest',
    'open-interest_bitfinex_btc_usdt_open_interest',
    'open-interest_bybit_all_symbol_open_interest'
] 
df = make_diff_change_feature(df, imp_open_interest_columns) # 변동성, 차분 피처 

# 3. 롱/숏 비율
df = make_liquidation_to_volume_ratio_feature(df, 'liquidations_deribit_all_symbol_long_liquidations', 'liquidations_deribit_all_symbol_short_liquidations',
    'taker-buy-sell-stats_binance_taker_buy_volume', 'taker-buy-sell-stats_binance_taker_sell_volume') # 롱/숏 비율

# 4. 청산/거래량 비율
df = make_liquidation_to_volume_ratio_feature(df,'liquidations_deribit_all_symbol_long_liquidations', 'liquidations_deribit_all_symbol_short_liquidations',
    'taker-buy-sell-stats_binance_taker_buy_volume','taker-buy-sell-stats_binance_taker_sell_volume')

# 5. # 청산된 USD 롱/숏 비율
df = make_liquidation_usd_ratio_feature(df,'liquidations_deribit_all_symbol_long_liquidations_usd','liquidations_deribit_all_symbol_short_liquidations_usd')

# 6. 펀딩 비율과 롱/숏 포지션 차이 곱
df = make_funding_rate_position_change_feature(df, 'funding-rates_okx_funding_rates', 'liquidations_deribit_all_symbol_long_liquidations', 
    'liquidations_deribit_all_symbol_short_liquidations')

# 7. 프리미엄 갭과 프리미엄 인덱스의 차이
df = make_premium_diff_feature(df, 'coinbase-premium-index_coinbase_premium_gap', 'coinbase-premium-index_coinbase_premium_index')

# 8. 해시레이트와 난이도 간의 비율
df = make_hashrate_to_difficulty_feature(df, 'hashrate_hashrate', 'difficulty_difficulty')

# 9. 공급 변화율
df = make_supply_change_rate_feature(df, 'supply_supply_new', 'supply_supply_total')

# 10. Rolling(이동평균) 피처
df = make_multiple_rolling_features(df, ['ID', 'target', '_type'] + ['year', 'month', 'week', 'day_of_week', 'hour'])

In [125]:
# 나머지 위의 피처엔지니어링 함수들을 실행한다.
conti_cols = list(set(conti_cols) - set(rolling_columns))

# 모든 수치형 컬럼에 대한 지수이동평균 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"_moving_avg_7"] = EMA(df, c, 7)

# 모든 수치형 컬럼에 대한 Wavelet transform을 계산해서 새로운 열로 할당
for c in conti_cols:
    df[c+"WT"] = WT(df, c)

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14343/1888448852.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"_moving_avg_7"] = EMA(df, c, 7)
/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14343/1888448852.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[c+"_moving_avg_7"] = EMA(df, c, 7)
/var/folders/4t/dyhc70gd7zjdwxdh7x7f77240000gn/T/ipykernel_14343/1888448852.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, wh

In [126]:
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

_target = df["target"]
df = df.ffill().fillna(-999).assign(target = _target)
# df = df.loc[:, ~df.columns.duplicated()]
# df = df.drop(columns=new_features)

train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

# Train

In [131]:
# 칼럼 그룹 정리  (추후 실험에서 특정 피처들 포함/미포함을 쉽게 관리하기 위함. 교집합 없음)

# 기존 칼럼
addresses_columns = [col for col in df.columns if 'addresses' in col] # addresses(주소) 관련 피처. 변수명에 'addresses'포함
block_columns = [col for col in df.columns if 'block-' in col] # block(블록) 관련 피처. 변수명에 'block-' 포함 
blockreward_columns = [col for col in df.columns if 'blockreward' in col] # blockreward 관련 피처
difficulty_columns = [col for col in df.columns if 'difficulty' in col] # difficulty 관련 피처
fees_columns = [col for col in df.columns if 'fees' in col] # fees 관련 피처
hashrate_columns = [col for col in df.columns if 'hashrate' in col] # hashrate 관련 피처
supply_columns = [col for col in df.columns if 'supply_supply' in col] # supply 관련 피처
tokens_columns = [col for col in df.columns if 'tokens' in col] # tokens 관련 피처
transactions_columns = [col for col in df.columns if 'transactions' in col] # transactions 관련 피처
utxo_columns = [col for col in df.columns if 'utxo' in col] # utxo 관련 피처
velocity_columns = [col for col in df.columns if 'velocity' in col] # velocity 관련 피처
premium_columns = [col for col in df.columns if 'premium' in col] # premium 관련 피처
funding_rates_columns = [col for col in df.columns if 'funding_rates' in col] # funding rates 관련 피처
long_liquidation_columns = [col for col in df.columns if 'long_liquidation' in col] # 롱 청산 관련 피처
short_liquidation_columns = [col for col in df.columns if 'short_liquidation' in col] # 숏 청산 관련 피처
open_interest_columns = [col for col in df.columns if 'open_interest' in col] # open-interest 관련 피처
taker_columns = [col for col in df.columns if 'taker' in col] # taker 관련 피처

# 피처 엔지니어링
baseline_extra_columns = ['liquidation_diff', 'liquidation_usd_diff', 'volume_diff', 'liquidation_diffg', 'liquidation_usd_diffg', 'volume_diffg', 'buy_sell_volume_ratio'] # 게시글에 있는 베이스라인 코드의 피처
date_columns = ['ID', 'year', 'month', 'week', 'day_of_week', 'hour']
open_interest_diff_columns = [col for col in df.columns if '_diff' in col]
open_interest_pct_columns =[col for col in df.columns if '_pct_change' in col]
domain_related_columns = ['long_short_ratio', 'liquidation_to_volume_ratio', 'liquidation_usd_ratio', 'funding_rate_position_change', 'premium_diff', 'hashrate_to_difficulty', 'supply_change_rate']
rolling_columns = [col for col in df.columns if 'rolling' in col] # rolling 관련 피처
wt_columns = [col for col in df.columns if 'WT' in col] # WT 관련 피처
ema_columns = [col for col in df.columns if col.endswith('_moving_avg_7')] # WT 관련 피처
shift_7_columns = [series.name for series in shift_list] # shift list 피처 

# category, continuous 열을 따로 할당해둠
category_cols: List[str] = ["liquidation_diffg", "liquidation_usd_diffg", "volume_diffg"]
conti_cols: List[str] = [col for col in df.columns if col not in category_cols + date_columns + rolling_columns + ['target','_type']] # category_cols, date_columns, rolling_columns 에 있는 것들을 제외

In [129]:
# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params (베이스라인 코드의 기본 파라미터임)
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/Users/yusolcho/opt/anaconda3/envs/datawrg/lib/python3.8/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4257990867579909, auroc: 0.6414069310418331


# 데이터 그룹 활용 예시: EMA, WT, Shift 를 뺴고 학습시켜보기


In [132]:
# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"] + wt_columns + ema_columns + shift_7_columns, axis = 1),     # 이렇게 train_df.drop에 해당 칼럼 그룹 리스트를 더해주면 학습에서 제외됨
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params (베이스라인 코드의 기본 파라미터임)
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

/Users/yusolcho/opt/anaconda3/envs/datawrg/lib/python3.8/site-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4246575342465753, auroc: 0.6405035143460982


In [ ]:
# performance 체크후 전체 학습 데이터로 다시 재학습
x_train = train_df.drop(["target", "ID"], axis = 1)
y_train = train_df["target"].astype(int)
train_data = lgb.Dataset(x_train, label=y_train)
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
)

In [ ]:
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [ ]:
submission = submission.assign(target = y_test_pred_class)
submission.to_csv("output.csv", index=False)